## Wavelet tests

In [ ]:
############
# Packages #
############
import os
import sys
import pickle
import numpy as np
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import seaborn as sns

import pywt
from scipy.signal import periodogram, fftconvolve, cwt

from typing import Dict, Union, List, Tuple, Any, Callable, Optional
pio.renderers.default = "plotly_mimetype+notebook"
################
#    Imports   #
################

root_path = Path(os.getcwd())
saving_path = root_path.joinpath("outputs")
# root_path should end by \nonlinear_ICA:
print(root_path)
sys.path.insert(0, str(root_path))

from src.data import (
    load_meta,
    add_superclasse,
    describ_raw_df,
    load_ecg,
    load_ecg_from_clean_data,
    plot_all_st
)

In [ ]:
################################
#  clean meta data  Loading    #
################################
with open(str(saving_path.joinpath("clean_data.pkl")), "rb") as f:
    df = pickle.load(f)

In [ ]:
############################
#  500hz signals  Loading  #
############################
signals = load_ecg_from_clean_data(df, root_path, patient_ids = df.loc[1:100,"patient_id"].tolist())

In [ ]:
plot_all_st(signals[0].T, clustering=None, title="<b>Signals 500Hz</b> patient 0")

### Wavelets mothers and fathers

In [ ]:
def add_fig(fig, signal, color, name):
    fig.add_trace(go.Scatter(y=signal, 
                 mode="lines", 
                 line=dict(
                     width=2,
                     color=color,
                 ),
                 opacity = 0.6,
                 name=name
                )
             )

def add_fig_continuous(level, w, fig, signal, color, wave):
    psi, x = w.wavefun(level=level)
    add_fig(fig, phi, color, f"{wave} wavelet continuous function psi")

def add_fig_discret_ortho(level, w, fig, signal, color, wave):
    (phi, psi, x) = w.wavefun(level=level)
    add_fig(fig, phi, color, f"{wave} scaling function phi")
    add_fig(fig, psi, color, f"{wave} wavelet function psi")

def add_fig_discret_biortho(level, w, fig, signal, color, wave):
    (phi_d, psi_d, phi_r, psi_r, x) = w.wavefun(level=level)
    add_fig(fig, phi_d, color, f"{wave} decomposition scaling phi_d")
    add_fig(fig, psi_d, color, f"{wave} decomposition wavelet psi_d")
    add_fig(fig, phi_r, color, f"{wave} reconstruction scaling phi_r")
    add_fig(fig, psi_r, color, f"{wave} reconstruction wavelet psi_r")

def plot_wave_family(level, type, iscontinous = False):
    lst_wave = pywt.wavelist(type)
    nwave = len(lst_wave)
    pal = sns.color_palette("Spectral", nwave).as_hex()
    fig = go.Figure(
        layout=go.Layout(
            height=600, 
            width=800, 
            template = "plotly_dark", 
            title = f"Wavelets {type}"
    ))
    
    for i in range(nwave):
        wave = lst_wave[i]
        w = pywt.DiscreteContinuousWavelet(wave)
        if iscontinous:
            # w = pywt.ContinuousWavelet(wave)
            add_fig_continuous(level, w, fig, signal, pal[i], wave)
        else:
            # w = pywt.Wavelet(wave)
            if w.orthogonal:
                add_fig_discret_ortho(level, w, fig, signal, pal[i], wave)
            else:
                add_fig_discret_biortho(level, w, fig, signal, pal[i], wave)

    fig.update_layout(title = w.family_name + f" code={type}  level = {level} ")
    fig.show()    

In [ ]:
level = 5

lst_types = pywt.families()
sep_continous = lst_types.index("gaus")

lst_discret = lst_types[:sep_continous]
lst_continuous = lst_types[sep_continous:]

for type in lst_discret:
    plot_wave_family(level, type, iscontinous = False)
    
for type in lst_continuous:
    plot_wave_family(level, type, iscontinous = True)

In [ ]:
for level in range(1,10,2 ):
    plot_wave_family(level, "db", iscontinous = False)
    
for level in range(1,10,2):
    plot_wave_family(level, "morl", iscontinous = True)

### Discrete Wavelet Transform

In [ ]:
def plot_signal(vec, title = "signal"):
    fig = px.line(vec, template = "plotly_dark", title = title)
    fig.show()

In [ ]:
signal = signals[0][:,0]

plot_signal(signal, title = "test signal: patient 0 elctrode 0")

A partir de la décomposition en ondellette on obtient l'approximation et le détail (ie le reste)